# 1. Imports

In [ ]:
import os
import tensorflow as tf
import numpy as np
from PIL import Image

np.set_printoptions(suppress=True, precision=2)

os.chdir("/Users/apple/Desktop/MIPT/Machine learning/nfl_impact_detection")

In [ ]:
from data_processing.Processor import load_detection_ds, Processor
from data_processing import image, bboxes
from utils.draw_bboxes import draw_bboxes

from models.fcos import build

Следующая ячейка перегоняет разметку из csv-файла в формат GMC. Ее достаточно запустить один раз в жизни. 

In [ ]:
if False:
    from data_processing.io import csv_to_json_markup

    csv_to_json_markup("data/image_labels.csv", "data/images_markup")

# 2. Preprocessing

Функция `load_detection_ds()` строит `tf.data.Dataset`.

In [ ]:
raw_ds = load_detection_ds("training/temp_dataset.yaml")

for sample in raw_ds:
    break

Объект `Processor` - это аналог нашего `BallfishTFwrapper`. При его инициализации указываешь список преобразований. Про аргумент `feature_keys` пока не думай...

In [ ]:
processor = Processor([
    image.LoadImage(),
    bboxes.LoadBboxes(n_bboxes=90),
    image.ResizeKeepRatio(300,400),
    image.Normalize(),
    bboxes.BuildFCOSTarget((300,400), [8])
], feature_keys=None)

processed_ds = raw_ds.map(processor)

In [ ]:
for sample in processed_ds.skip(5):
    break
    
sample.keys()

In [ ]:
img = draw_bboxes((sample["img"].numpy() * 255).astype(np.uint8), sample["bboxes"].numpy(), show_desc=False)
Image.fromarray(img)

In [ ]:
print(sample["target0"].numpy().shape)
Image.fromarray((sample["target0"].numpy()[:,:,0] * 255).astype(np.uint8)).resize((400,300))

# 3. Model

In [ ]:
model = build.build_model()

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=True, show_layer_names=False, expand_nested=False
)